In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import pandas as pd
import numpy as np
from nlpaug.util import Action


import simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import wandb
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('messages_labels_1000_1234.csv')

In [ ]:
from sklearn.utils import shuffle
from tqdm import tqdm
def augment_text(df, samples = 300, pr = 0.2, class_name = 1, mode = 'syn'):
    syn_aug = naw.SynonymAug(aug_src = 'wordnet', stopwords = stop_words)
    syn_aug.aug_p=pr
    
    del_aug = naw.RandomWordAug(stopwords = '[scrubbed]')

    swap_aug = naw.RandomWordAug(action="swap")
    
    new_text=[]
    interaction = []
    conv_ids = []
    mess_ids = []
    ##dropping samples from validation
    df_n=df[df.convo_stage == class_name].reset_index(drop=True)
    ## data augmentation loop
    for i in tqdm(np.random.randint(0,len(df_n),samples)):
        text = df_n.iloc[i]['message']
        person = df_n.iloc[i]['interaction']
        conv_id = df_n.iloc[i]['conversation_id']
        mess_id = df_n.iloc[i]['message_id']
        if mode == 'syn':           
            augmented_text = syn_aug.augment(text)
            
        elif mode == 'del':           
            augmented_text = del_aug.augment(text)
        
        elif mode == 'swap':
            augmented_text = swap_aug.augment(text)
            
        new_text.append(augmented_text)
        interaction.append(person)
        conv_ids.append(conv_id)
        mess_ids.append(mess_id)
        
        
    ## dataframe
    new=pd.DataFrame({'message':new_text, 'convo_stage': class_name, 'interaction': interaction, 
                      'conversation_id':conv_ids, 'message_id':mess_ids})
    new['encoded_message'] = "[" + new.interaction + "]" + ' ' + new.message
    return new

In [ ]:
class_1_aug_syn = augment_text(df, samples = 200, class_name = 1, mode = 'syn')
class_1_aug_del = augment_text(df, samples = 40, class_name = 1, mode = 'del')
class_1_aug_swap = augment_text(df, samples = 40, class_name = 1, mode = 'swap')
class_1_aug = pd.concat([class_1_aug_del, class_1_aug_syn, class_1_aug_swap], ignore_index=True)

In [ ]:
class_2_aug_syn = augment_text(df, samples = 600, class_name = 2, mode = 'syn')
class_2_aug_del = augment_text(df, samples = 120, class_name = 2, mode = 'del')
class_2_aug_swap = augment_text(df, samples = 120, class_name = 2, mode = 'swap')
class_2_aug = pd.concat([class_2_aug_del, class_2_aug_syn, class_2_aug_swap], ignore_index=True)

In [ ]:
class_3_aug_syn = augment_text(df, samples = 600, class_name = 3, mode = 'syn')
class_3_aug_del = augment_text(df, samples = 120, class_name = 3, mode = 'del')
class_3_aug_swap = augment_text(df, samples = 120, class_name = 3, mode = 'swap')
class_3_aug = pd.concat([class_3_aug_del, class_3_aug_syn, class_3_aug_swap], ignore_index=True)

In [ ]:
class_4_aug_syn = augment_text(df, samples = 900, class_name = 4, mode = 'syn')
class_4_aug_del = augment_text(df, samples = 180, class_name = 4, mode = 'del')
class_4_aug_swap = augment_text(df, samples = 180, class_name = 4, mode = 'swap')
class_4_aug = pd.concat([class_4_aug_del, class_4_aug_syn, class_4_aug_swap], ignore_index=True)

In [ ]:
class_5_aug_syn = augment_text(df, samples = 200, class_name = 5, mode = 'syn')
class_5_aug_del = augment_text(df, samples = 40, class_name = 5, mode = 'del')
class_5_aug_swap = augment_text(df, samples = 40, class_name = 5, mode = 'swap')
class_5_aug = pd.concat([class_5_aug_del, class_5_aug_syn, class_5_aug_swap], ignore_index=True)

In [ ]:
aug_df = pd.concat([class_5_aug, class_4_aug, class_3_aug, class_2_aug,class_1_aug], ignore_index=True)

In [ ]:
aug_df.to_csv('aug_df.csv')